# Road Following Live (with camera)

In [2]:
import torch
import sys
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('/home/jetson/github/tfg-amariscal/models/road_following_model_trt_3.pth'))
print("Model loaded")

Model loaded


Create the racecar class

In [3]:
from jetracer.my_racecar import MyRacecar
import Jetson.GPIO as GPIO
#from motorsJetson import Motors

# Left motors
ENA = 33
IN1 = 21
IN2 = 22

# Right motors
ENB = 32
IN3 = 26
IN4 = 24

# 50 Hz
FREQUENCY = 50

car = MyRacecar()
#GPIO.setmode(GPIO.BOARD)
#GPIO.setwarnings(False)

#motors = Motors(ENA, IN1, IN2, ENB, IN3, IN4, FREQUENCY)
print("Car loaded")

Car loaded


Create the camera class.

In [4]:
from jetcam.usb_camera import USBCamera

camera = USBCamera(width=224, height=224, capture_width=640, capture_height=480, capture_device=0)
print("Camera loaded")

Capture device is 0
Camera loaded


In [5]:
import cv2
import ipywidgets
import threading

state_widget = ipywidgets.ToggleButtons(options=['On', 'Off'], description='Camera', value='On')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])
print("live_execution_widget loaded")

live_execution_widget loaded


Before letting your car go, let's prepare slide bars for gains and offset, so that you can adjust them during the runtime.

In [6]:
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets

network_output_slider = widgets.FloatSlider(description='Network Output', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_gain_slider  = widgets.FloatSlider(description='Steering Gain', min=-10.0, max=10.0, value=1, step=1, orientation='horizontal', layout={'width': '300px'})
steering_bias_slider  = widgets.FloatSlider(description='Steering Bias', min=-0.5, max=0.5, value=0.0, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = widgets.FloatSlider(description='Steering', min=-30.0, max=30.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = widgets.FloatSlider(description='Throttle', min=15, max=50, value=25, step=1, orientation='vertical')


steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (car, 'steering_offset'))
#steering_value_link  = traitlets.link((steering_value_slider, 'value'), (car, 'steering'))
#throttle_slider_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))

display(
    widgets.HBox(
        [widgets.VBox([network_output_slider,
                       widgets.Label(value="X"),
                       steering_gain_slider,
                       widgets.Label(value="+"),
                       steering_bias_slider,
                       widgets.Label(value="||"), 
                       steering_value_slider], layout=Layout(
                                                    align_items='center'
                                                        )
                     ), 
         live_execution_widget,
         throttle_slider]
    )
)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [7]:
from utils import preprocess
from jetcam.utils import bgr8_to_jpeg

def update(change):
    global blocked_slider, robot
    new_image = change['new'] 
    
    image = preprocess(new_image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    y = float(output[0])
    
    network_output_slider.value = x
    steering = x * steering_gain_slider.value + steering_bias_slider.value
    
    #if(steering<-1.0):
     #   steering_value_slider.value = -1.0
    #elif(steering>1.0):
    #    steering_value_slider.value = 1.0
    #else:
    steering_value_slider.value = steering 
    car._on_steering(steering)
    car._on_throttle(throttle_slider.value)
    print("Thottle value: " + str(throttle_slider.value))
    
    if(state_widget.value == 'On'):
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))  
        prediction = new_image.copy()
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
        
update({'new': camera.value})  # we call the function once to initialize

Forward: -0.122314453125 Throttle: 0
Thottle value: 25.0


In [8]:
camera.observe(update, names='value') 

In [9]:
camera.running = True

Forward: 0.168212890625 Throttle: 25.0
Thottle value: 25.0
Forward: 0.16748046875 Throttle: 25.0
Thottle value: 25.0
Forward: 0.1639404296875 Throttle: 25.0
Thottle value: 25.0
Forward: 0.1622314453125 Throttle: 25.0
Thottle value: 25.0
Forward: 0.16162109375 Throttle: 25.0
Thottle value: 25.0
Forward: 0.165771484375 Throttle: 25.0
Thottle value: 25.0
Forward: 0.16552734375 Throttle: 25.0
Thottle value: 25.0
Forward: 0.1678466796875 Throttle: 25.0
Thottle value: 25.0
Forward: 0.1617431640625 Throttle: 25.0
Thottle value: 25.0
Forward: 0.1541748046875 Throttle: 25.0
Thottle value: 25.0
Forward: 0.173828125 Throttle: 25.0
Thottle value: 25.0
Forward: 0.1640625 Throttle: 25.0
Thottle value: 25.0
Forward: 0.1607666015625 Throttle: 25.0
Thottle value: 25.0
Forward: 0.161865234375 Throttle: 25.0
Thottle value: 25.0
Forward: 0.1583251953125 Throttle: 25.0
Thottle value: 25.0
Forward: 0.1588134765625 Throttle: 25.0
Thottle value: 25.0
Forward: 0.156005859375 Throttle: 25.0
Thottle value: 25.0


In [ ]:
camera.close

In [ ]:
car.__del__()